In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install sacrebleu

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/c2/d6/ff9baefc8fc679dcd9eb21b29da3ef10c81aa36be630a7ae78e4611588e1/evaluate-0.4.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00a 0:00:010m
  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/df/d5/f07d3c37bd98db883330276d77e7b04b6c50564c68fb95a76e05422a2850/sacrebleu-2.4.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/9b/fb/a70a4214956182e0d7a9099ab17d50bfcba1056188e9b14f35b9e2b62a0d/portalocker-2.10.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.1 MB/s eta 0:00:00 0:00:01


link to wandb project: https://wandb.ai/FinalProject_/T5/runs/7jbnfz5h?nw=nwusermoudjaramina2001 
link to project (with and without optimization): https://wandb.ai/FinalProject_/T5?nw=nwusermoudjaramina2001 

In [3]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "T5"
os.environ["WANDB_NOTES"] = "Fine tune t5 without optimization"
os.environ["WANDB_NAME"] = "finetune-t5-base-on-opus100-Ar2En-without-optimization"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading OPUS Books dataset

Here we start by loading the English-French subset of the OPUS dataset

In [6]:
from datasets import load_dataset

books=load_dataset("opus100", "ar-en", split="train[:7000]")
books

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset opus100 downloaded and prepared to /root/.cache/huggingface/datasets/opus100/ar-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704. Subsequent calls will reuse this data.


Dataset({
    features: ['translation'],
    num_rows: 7000
})

In [7]:
books=books.train_test_split(test_size=0.7)
books["train"]

Dataset({
    features: ['translation'],
    num_rows: 2100
})

# Prerpocessing

In [8]:
from transformers import AutoTokenizer

model_name="UBC-NLP/AraT5v2-base-1024"
tokenizer=AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [9]:
source_lang="ar"
target_lang="en"
prefix="translate Arabic to English: "

def preprocess_function(examples):
    inputs =[prefix+example[source_lang] for example in examples["translation"]]
    targets=[example[target_lang] for example in examples["translation"]]
    model_inputs=tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs


tokenized_books=books.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)
print(data_collator)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


DataCollatorForSeq2Seq(tokenizer=T5TokenizerFast(name_or_path='UBC-NLP/AraT5v2-base-1024', vocab_size=110100, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_

# Evaluate



In [11]:
import numpy as np
import evaluate

metric_bleu = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds=[pred.strip() for pred in preds]
    labels=[[label.strip()] for label in labels]
    
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels=eval_preds
    if isinstance(preds, tuple):
        preds=preds[0]
    
    decoded_preds=tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    labels=np.where(labels !=-100, labels, tokenizer.pad_token_id)
    decoded_labels=tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds,decoded_labels=postprocess_text(decoded_preds, decoded_labels)
    
    result=metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    result={"bleu":result["score"]}
    
    prediction_lens=[np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in preds]
    result["gen_len"]=np.mean(prediction_lens)
    result={k: round(v,4) for k, v in result.items()}
    return result




# Training

In [12]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
print(model.config)

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

T5Config {
  "_name_or_path": "UBC-NLP/AraT5v2-base-1024",
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size": 110208
}



In [13]:
model

T5ForConditionalGeneration(
  (shared): Embedding(110208, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(110208, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo

In [14]:
training_args=Seq2SeqTrainingArguments(
    output_dir=os.getenv("WANDB_NAME"),
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir=os.getenv("WANDB_NAME")+"/logs",
    logging_strategy="epoch",
    logging_steps=500,
    load_best_model_at_end=True,
    learning_rate=2e-5,
    per_device_train_batch_size= 5,
    per_device_eval_batch_size= 5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs= 18,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    report_to="wandb",
    run_name=os.getenv("WANDB_NAME"),
)

trainer=Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Currently logged in as: moudjaramina2001 (FinalProject_). Use `wandb login --relogin` to force relogin


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,10.144800,3.925647,2.833500,9.498800
2,4.982200,3.575952,4.900100,10.332900
3,4.420000,3.403652,5.697300,10.301000
4,4.141400,3.305676,6.522400,10.555900
5,3.945100,3.216878,7.409000,10.757100
6,3.797200,3.175907,8.144500,10.590800
7,3.668700,3.134008,8.246000,10.745100
8,3.549400,3.109762,8.565600,10.761600
9,3.474800,3.074903,9.052000,10.879800
10,3.394500,3.072485,9.322300,10.679400


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars

TrainOutput(global_step=3780, training_loss=3.9419209152302415, metrics={'train_runtime': 8026.6341, 'train_samples_per_second': 4.709, 'train_steps_per_second': 0.471, 'total_flos': 2774648751851520.0, 'train_loss': 3.9419209152302415, 'epoch': 18.0})

In [16]:

eval_results=trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 20.17


# Inference

In [17]:
from transformers import pipeline

text="translate English to French: المملكة العربية السعودية"
translator=pipeline("translation",model=os.getenv("WANDB_NAME"))
translator(text)

[{'translation_text': 'The Kingdom of Saudi Arabia'}]

In [18]:
tokenizer.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(os.getenv("WANDB_NAME"))

'https://huggingface.co/yasmineee/finetune-t5-base-on-opus100-Ar2En-without-optimization/tree/main/'

# Inference with PyTorch

In [19]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(os.getenv("WANDB_NAME"))
inputs=tokenizer(text, return_tensors="pt").input_ids

In [20]:
from transformers import AutoModelForSeq2SeqLM

model=AutoModelForSeq2SeqLM.from_pretrained(os.getenv("WANDB_NAME"))
outputs=model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [22]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'I have a Saudi arabia!'